## 데이터 파악 (훈련데이터,테스트 데이터 브랜드명 추출)
### 추출시 공통된 브랜드가없다.

In [3]:
import os
import json

def extract_brand_names_from_first_item(directory_path):
    brand_names = []

    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                    if isinstance(data, list) and len(data) > 0:
                        first_item = data[0]
                        brand_name = first_item.get("JNG_INFO", {}).get("BRAND_NM")
                        brand_num = first_item.get("JNG_INFO", {}).get("JNG_IFRMP_SN")
                        if brand_name:
                            brand_names.append((filename, brand_name))
                        else:
                            print(f"[경고] '{filename}'에서 BRAND_NM을 찾을 수 없습니다.")
                    else:
                        print(f"[경고] '{filename}'은 리스트가 아니거나 비어 있습니다.")

            except Exception as e:
                print(f"[에러] '{filename}' 처리 중 오류 발생: {e}")

    return brand_names

def save_brand_names_to_txt(brand_list, output_path):
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            for filename, brand in brand_list:
                f.write(f"{filename}: {brand}\n")
        print(f"[저장 완료] {output_path}")
    except Exception as e:
        print(f"[에러] 파일 저장 중 오류 발생: {e}")


# 사용 예시
if __name__ == "__main__":
    train_directory = ("/home/sm7540/workspace/franchise_rag/data/train")
    test_directory = ("/home/sm7540/workspace/franchise_rag/data/test")
    train_brand_nm =  extract_brand_names_from_first_item(train_directory)
    test_brand_nm =  extract_brand_names_from_first_item(test_directory)
    # 텍스트 파일로 저장
    save_brand_names_to_txt(train_brand_nm, "train_brands.txt")
    save_brand_names_to_txt(test_brand_nm, "test_brands.txt")
    # 브랜드명만 추출해서 set으로 변환
    train_brands = set([brand for _, brand in train_brand_nm])
    test_brands = set([brand for _, brand in test_brand_nm])

    # 공통 브랜드명 추출
    common_brands = train_brands.intersection(test_brands)

    # 결과 출력
    print("\n공통된 브랜드명:")
    for brand in sorted(common_brands):
        print(f"- {brand}")



[저장 완료] train_brands.txt
[저장 완료] test_brands.txt

공통된 브랜드명:


### 사전지식베이스 구축

In [ ]:
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain.docstore.document import Document
# from langchain_chroma import Chroma
# from config import Settings
# import json
# import os

# settings = Settings(JSON_PATH="/home/sm7540/workspace/franchise_rag/data/train")

# # HuggingFace 임베딩 모델 초기화 (KURE-v1)
# try:
#     embeddings = HuggingFaceEmbeddings(
#         model_name=settings.EMBEDDING_MODEL_PATH,  # 로컬 경로 사용
#         model_kwargs={
#             'device': 'cuda'
#         }
#     )
#     print("로컬 임베딩 모델 로드 성공")
# except Exception as e:
#     print("로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다")
#     # 실패 시 온라인 모델로 폴백
#     embeddings = HuggingFaceEmbeddings(
#         model_name=settings.EMBEDDING_MODEL_NAME,
#         model_kwargs={'device': 'cuda'}
#     )

# print(f"임베딩 모델{settings.EMBEDDING_MODEL_NAME} 로딩 완료")

# # 벡터 저장소 디렉토리 생성
# vector_db_path = settings.VECTOR_DB_PATH
# os.makedirs(vector_db_path, exist_ok=True)

# # 테스트 JSON 파일 로드
# try:
#     with open(settings.JSON_PATH, 'r', encoding='utf-8') as file:
#         contracts_data = json.load(file)
#     print(f"총 {len(contracts_data)} 개의 계약서 데이터를 로드했습니다.")
# except FileNotFoundError:
#     print("파일을 찾을 수 없습니다: ./test.json")
#     contracts_data = []
# except json.JSONDecodeError:
#     print("JSON 파일 파싱 중 오류가 발생했습니다.")
#     contracts_data = []
#     exit()

# # 문서 객체 생성
# documents = []

# for contract in contracts_data:
#     doc_id = f"{contract['LRN_DTIN_MNNO']}_{contract['CHNK_NO']}"

#     ## 브랜드 정보,가맹 정보
#     metadata = {
#         "ID": contract["LRN_DTIN_MNNO"],
#         "source": doc_id,
#         "brand": contract["JNG_INFO"]["BRAND_NM"],
#         "company": contract["JNG_INFO"]["JNGHDQRTRS_CONM_NM"],
#         "year": contract["JNG_INFO"]["JNG_BIZ_CRTRA_YR"]
#     }
    
#     # Content 구성: JSON 객체를 문자열로 변환
#     content = [{
#         "topic": contract["ATTRB_INFO"]["KORN_UP_ATRB_NM"],
#         "sub_topic": contract["ATTRB_INFO"]["KORN_ATTRB_NM"],
#         "contents": contract["QL"]["EXTRACTED_SUMMARY_TEXT"] 
#     }]
#     content_str = json.dumps(content, ensure_ascii=False)  # 리스트를 JSON 문자열로 변환

#     # LangChain Document 객체 생성
#     doc = Document(page_content=content_str, metadata=metadata)
#     documents.append(doc)

# print(f"{len(documents)}개의 문서 객체 생성 완료")

# # Chroma 벡터 스토어 생성
# try:
#     vector_store = Chroma.from_documents(
#         documents=documents,
#         embedding=embeddings,
#         collection_name="contracts_collection",
#         persist_directory=vector_db_path
#     )
#     print(f"벡터 스토어 생성 완료. 저장 경로: {vector_db_path}")
    
# except Exception as e:
#     print(f"벡터 스토어 생성 중 오류 발생: {str(e)}")

로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다
임베딩 모델 nlpai-lab/KURE-v1 로딩 완료
총 57 개의 계약서 데이터를 로드했습니다.
57개의 요약 기반 문서 객체 생성 완료
❌ 벡터 스토어 생성 중 오류 발생: Expected metadata value to be a str, int, float or bool, got [{'QUESTION': '㈜틴트어카코리아의 상호명은 무엇인가요?', 'ANSWER': '상호명은 ㈜틴트어카코리아입니다.'}, {'QUESTION': '틴트어카(Tint a Car)의 대표자는 누구인가요?', 'ANSWER': '대표자는 정수희입니다.'}, {'QUESTION': '㈜틴트어카코리아의 법인 등록 번호는 무엇인가요?', 'ANSWER': '법인 등록 번호는 134511-0132263입니다.'}, {'QUESTION': '틴트어카(Tint a Car)의 사업자 등록일은 언제인가요?', 'ANSWER': '사업자 등록일은 2008년 11월 6일입니다.'}, {'QUESTION': '㈜틴트어카코리아의 주소는 어떻게 되나요?', 'ANSWER': '주소는 경기도 용인시 *** *** **** ** ***** ** *****입니다.'}, {'QUESTION': '틴트어카(Tint a Car)의 대표 전화번호는 무엇인가요?', 'ANSWER': '대표 전화번호는 1644-8468입니다.'}, {'QUESTION': '㈜틴트어카코리아의 설립 등기일은 언제인가요?', 'ANSWER': '설립 등기일은 2008년 11월 6일입니다.'}, {'QUESTION': '틴트어카(Tint a Car)의 대표 팩스 번호는 무엇입니까?', 'ANSWER': '대표 팩스 번호는 031-334-1655입니다.'}, {'QUESTION': '㈜틴트어카코리아의 사업자 등록 번호는 무엇인가요?', 'ANSWER': '사업자 등록 번호는 142-81-16917입니다.'}] which is a list in upsert.

Try filtering c

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from config import Settings
import json
import os

settings = Settings(JSON_PATH="/home/sm7540/workspace/franchise_rag/data/train/1056757501.json")

# HuggingFace 임베딩 모델 초기화
try:
    embeddings = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_PATH,
        model_kwargs={'device': 'cuda'}
    )
    print("로컬 임베딩 모델 로드 성공")
except Exception:
    print("로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다")
    embeddings = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cuda'}
    )

print(f"임베딩 모델 {settings.EMBEDDING_MODEL_NAME} 로딩 완료")

# 벡터 저장소 디렉토리 생성
vector_db_path = settings.VECTOR_DB_PATH
os.makedirs(vector_db_path, exist_ok=True)

# JSON 데이터 로드
try:
    with open(settings.JSON_PATH, 'r', encoding='utf-8') as file:
        contracts_data = json.load(file)
    print(f"총 {len(contracts_data)} 개의 계약서 데이터를 로드했습니다.")
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {settings.JSON_PATH}")
    contracts_data = []
    exit()
except json.JSONDecodeError:
    print("JSON 파일 파싱 중 오류가 발생했습니다.")
    contracts_data = []
    exit()

# 문서 객체 생성
documents = []
ids = []

for contract_idx, contract in enumerate(contracts_data):
    summary_text = contract.get("QL", {}).get("ABSTRACTED_SUMMARY_TEXT", "").strip()
    qas = contract.get("QL", {}).get("QAs", [])

    # 메타데이터 구성
    metadata = {
        "CHNK_NO": contract["CHNK_NO"],
        "SMRT_CHNK_NO": contract["SMRT_CHNK_NO"],
        "JNG_BIZ_CRTRA_YR": contract["JNG_INFO"]["JNG_BIZ_CRTRA_YR"],
        "JNGHDQRTRS_CONM_NM": contract["JNG_INFO"]["JNGHDQRTRS_CONM_NM"],
        "BRAND_NM": contract["JNG_INFO"]["BRAND_NM"],
        "JNG_IFRMP_SN": contract["JNG_INFO"]["JNG_IFRMP_SN"],
        "ATTRB_MNNO": contract["ATTRB_INFO"]["ATTRB_MNNO"],
        "KORN_ATTRB_NM": contract["ATTRB_INFO"]["KORN_ATTRB_NM"],
        "UP_ATTRB_MNNO": contract["ATTRB_INFO"]["UP_ATTRB_MNNO"],
        "KORN_UP_ATRB_NM": contract["ATTRB_INFO"]["KORN_UP_ATRB_NM"],
        "source": f"{contract['JNG_INFO']['JNG_IFRMP_SN']}.json",
        "QAs": json.dumps(qas, ensure_ascii=False)  # 문자열로 저장  # QA 전체 포함
    }
    
    doc_id = f"{contract['JNG_INFO']['JNG_IFRMP_SN']}_{contract_idx}"
    documents.append(Document(page_content=summary_text, metadata=metadata))
    ids.append(doc_id)

print(f"{len(documents)}개의 요약 기반 문서 객체 생성 완료")

# Chroma 벡터 스토어 생성
try:
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name="contracts_summary_collection",
        persist_directory=vector_db_path,
        ids=ids
    )
    print(f"✅ 벡터 스토어 생성 완료. 저장 경로: {vector_db_path}")
except Exception as e:
    print(f"❌ 벡터 스토어 생성 중 오류 발생: {str(e)}")


로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다
임베딩 모델 nlpai-lab/KURE-v1 로딩 완료
총 57 개의 계약서 데이터를 로드했습니다.
57개의 요약 기반 문서 객체 생성 완료
✅ 벡터 스토어 생성 완료. 저장 경로: ./vector_db/franchise


In [ ]:
from franchise import GeminiFranchiseService
from config import Settings,settings



# 서비스 초기화
GEMINI_API_KEY = settings.GEMINI_API_KEY

# 서비스 초기화
rag_service = GeminiFranchiseService(
    api_key=GEMINI_API_KEY,
    config={
        "vector_db_path": settings.VECTOR_DB_PATH,
        "model_name": settings.MODEL_NAME,
        "embedding_model_path": settings.EMBEDDING_MODEL_PATH
    }
)


qa_vectorstore = rag_service.chroma_vectorstore



# 기본적인 similarity search 사용
    
    # retriever = chroma_vectorstore.as_retriever(
    #     search_kwargs={
    #         "k": 5,  # 상위 5개 결과
    #         "filter": {
    #             "ATTRB_MNNO": "101007501"
    #         }
    #     }
    # )

   
    
    # # 테스트 질문
    # test_questions = [
    #     "서영에프앤비의 주소는 어디인가요?",
    #     "(주)서영에프앤비에서 운영하는 브랜드는?",
    #     # "피자헛의 창업 비용은 얼마인가요?" # 데이터에 없는 질문
    # ]
    
    # # 질문 답변 테스트
    # for i, question in enumerate(test_questions, 1):
    #     print(f"\n--- 질문 {i}: {question} ---")
    #     answer = rag_service.answer_question(question)
    #     print(f"답변: {answer}")

/home/sm7540/workspace/franchise_rag/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 16:20:54,834 - franchise - INFO - 로컬 임베딩 모델 로드 중: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 16:20:56,557 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 16:20:56,558 - franchise - ERROR - 로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다: Path C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9 not found
2025-05-11 16:20:56,558 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransforme

In [ ]:
from config import Settings
from fewshot_franchise import GeminiFewShotFranchiseService,load_prompt_templates,build_prompt_from_template

settings = Settings(DEVICE="cuda")

rag_service = GeminiFewShotFranchiseService(
    api_key=settings.GEMINI_API_KEY,
    config={
        "vector_db_path": settings.VECTOR_DB_PATH,
        "model_name": settings.MODEL_NAME,
        "embedding_model_path": settings.EMBEDDING_MODEL_PATH,
        "device":settings.DEVICE,
        "vectorstore_search_k":1
    }
)



/home/sm7540/workspace/franchise_rag/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 20:26:36,844 - franchise - INFO - 로컬 임베딩 모델 로드 중: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 20:26:38,590 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 20:26:38,590 - franchise - ERROR - 로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다: Path C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9 not found
2025-05-11 20:26:38,591 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransforme

None


In [ ]:
from langchain.docstore.document import Document

query = "서영에프앤비의 주소는 어디인가요?"



qa_vectorstore = rag_service.qa_vectorstore
vectorstore = rag_service.chroma_vectorstore

context_docs = rag_service.retrieve_context(vectorstore, query)
context = context_docs[0].page_content
attrb_mnno = context_docs[0].metadata.get("ATTRB_MNNO")



if qa_vectorstore:
    qa_docs = rag_service.retrieve_context(rag_service.qa_vectorstore, query, filter={"ATTRB_MNNO": attrb_mnno})

    if not qa_docs:
        print("⚠ 필터 조건에 맞는 결과가 없어 전체 문서에서 검색을 시도합니다.")
        qa_docs = rag_service.retrieve_context(rag_service.qa_vectorstore, query)


    prompt = rag_service.build_prompt_from_template(
        rag_service.prompt_template["fewshot_template"],
        query,
        context,
        docs=qa_docs
    )
else:
    prompt = rag_service.build_prompt_from_template(
        rag_service.prompt_template["basic_template"],
        query,
        context
    )


⚠ 필터 조건에 맞는 결과가 없어 전체 문서에서 검색을 시도합니다.


In [7]:
prompt




"당신은 정보공개서를 기반으로 가맹본부에 대한 질문에 정확하고 사실 기반의 답변을 제공하는 AI 어시스턴트입니다.\n\n다음은 예시 질문과 답변입니다:\nQ: 가맹본부 원스F&B의 대표자는 누구인가요?\nA: 대표자는 조창덕입니다.\n\nQ: 브랜드 '으니의 수라간'의 가맹사업 경영기간은 어떻게 되나요?\nA: 가맹사업 경영기간은 2011년 8월 10일부터 현재까지 진행되고 있습니다.\n\nQ: 가맹본부 원스F&B의 주된 사무소는 어디에 위치하고 있나요?\nA: 현재 주된 사무소는 경상북도 문경시 ****** *******에 위치하고 있습니다.\n\n[참고 문서]\n가맹본부: (주)미지에듀\n 브랜드: SBS아카데미뷰티스쿨\n 목차: 2) 영업지역 설정기준\n당사는 영업지역을 다음과 같이 정하고 있습니다.<table><tr><td>설정 기준 </td><td>설정방법 </td></tr><tr><td>서울시 각 구 1지점 </td><td>- 설정기준에 의해 가맹희망자와 협의 후 설정 - 별지에 영업지역 표시(지도별첨을 원칙으로 함) </td></tr><tr><td>광역시 2지점 </td></tr><tr><td>시 단위 1지점 </td></tr></table>\n\n[질문]\n서영에프앤비의 주소는 어디인가요?\n\n[답변]\n"

In [ ]:

query = "클랩피자 가맹점으로 가맹계약을 체결하면 어떤 지식재산권이 부여되나요?"
context = "가맹본부: (주)클랩피자\n 목차: 9. 사용을 허용하는 지식재산권\n귀하가 당사의 클랩피자가맹점으로 가맹계약을 체결할 경우에 귀하에게 부여하는 지식재산권의 정보는 다음과 같습니다.<table><tr><td>명칭</td><td>권리내용</td><td>등록 및 등록신청 여부(일자)</td><td>출원 및 등록번호 </td><td>출원인</td><td>존속기간 만료일</td></tr><tr><td></td><td>상표권</td><td>등록 2021.02.26.</td><td>제40-1697779호</td><td>이준범</td><td>2031.02.26.</td></tr></table>"
results = chroma_vectorstore.similarity_search(
    query,
    k=1,
    filter={"ATTRB_MNNO": "22"}
)

if not results:
    print("⚠ 필터 조건에 맞는 결과가 없어 전체 문서에서 검색을 시도합니다.")
    results = chroma_vectorstore.similarity_search(query, k=1)

prompt = build_prompt_with_examples_and_custom_context(results,query,context)
print(prompt)



# 결과 출력
for i, doc in enumerate(results, 1):
    print(f"🔹 결과 {i}")
    print("📄 Content:")
    print(doc.page_content)
    print("🧾 Metadata:")
    for key, value in doc.metadata.items():
        print(f"{key}: {value}")
    print("=" * 50)


⚠ 필터 조건에 맞는 결과가 없어 전체 문서에서 검색을 시도합니다.
당신은 정보공개서를 기반으로 가맹본부에 대한 질문에 정확하고 사실 기반의 답변을 제공하는 AI 어시스턴트입니다.

아래는 질문-답변 형식 예시입니다:

Q: 클랩피자의 상표권 사용 시 주의해야 할 점은 무엇인가요?
A: 클랩피자의 상표권 사용 시 귀하가 허용된 범위를 벗어나 사용할 경우 민형사상 책임을 질 수 있으므로 주의해야 합니다.

Q: 클랩피자의 상표권은 누구에게서 사용 허락을 받았나요?
A: 클랩피자는 상표권 권리자로부터 상표권 사용 허락을 받았습니다.

---

[참고 문서 내용]
가맹본부: (주)클랩피자
 목차: 9. 사용을 허용하는 지식재산권
귀하가 당사의 클랩피자가맹점으로 가맹계약을 체결할 경우에 귀하에게 부여하는 지식재산권의 정보는 다음과 같습니다.<table><tr><td>명칭</td><td>권리내용</td><td>등록 및 등록신청 여부(일자)</td><td>출원 및 등록번호 </td><td>출원인</td><td>존속기간 만료일</td></tr><tr><td></td><td>상표권</td><td>등록 2021.02.26.</td><td>제40-1697779호</td><td>이준범</td><td>2031.02.26.</td></tr></table>

---

[사용자 질문]
Q: 클랩피자 가맹점으로 가맹계약을 체결하면 어떤 지식재산권이 부여되나요?
A:
🔹 결과 1
📄 Content:
가맹본부는 (주)클랩피자로, 브랜드명은 클랩피자입니다. 사용자가 허가된 범위를 초과하여 지식재산권을 사용할 경우 민형사상 책임이 발생할 수 있으며, 당사는 상표권 사용허락을 받았습니다.
🧾 Metadata:
ATTRB_MNNO: 109007501
BRAND_NM: 클랩피자
CHNK_NO: 1
JNGHDQRTRS_CONM_NM: (주)클랩피자
JNG_BIZ_CRTRA_YR: 2024
JNG_IFRMP_SN: 2451867501
KORN_ATTRB_NM: 9. 사용을 허용하는 지식재산권
KORN_U